In [1]:
from datasets import load_dataset
from tqdm import tqdm
import json

/home/tavandai/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df = load_dataset('dataset/vn-legal-conversation')

Generating train split: 31433 examples [00:02, 14507.84 examples/s]
Generating validation split: 1746 examples [00:00, 16515.02 examples/s]
Generating test split: 1747 examples [00:00, 14324.83 examples/s]


In [5]:
df

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 31433
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 1746
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 1747
    })
})

In [6]:
df["train"][0]

{'Unnamed: 0': 2879,
 'Question': 'Mẹ tôi và dượng tôi ở với nhau gần 10 năm nhưng không đăng ký kết hôn. Nay dượng tôi phản bội mẹ tôi, có vợ mới và muốn chia đôi số tài sản, trong đó tiền vốn là của tôi bỏ ra cho mẹ tôi làm ăn. Ông ta đòi làm đơn kiện nếu mẹ tôi không đồng ý chia đôi số tài sản hiện tại. Trường hợp này phải giải quyết như thế nào? (Số tiền tôi đưa mẹ làm ăn không có giấy tờ gì chứng minh cả)',
 'Answer': 'Khoản 1 Điều 9 Luật hôn nhân và gia đình 2014 quy định: “Việc kết hôn phải được đăng ký và do cơ quan nhà nước có thẩm quyền thực hiện theo quy định của Luật này và pháp luật về hộ tịch. Việc kết hôn không được đăng ký theo quy định tại khoản này thì không có giá trị pháp lý”. Trường hợp bạn nêu, mẹ và dượng bạn sống chung nhưng không đăng ký kết hôn nên giữa mẹ và dượng bạn không tồn tại quan hệ hôn nhân. Cụ thể, theo quy định tại khoản 1 Điều 14 Luật này thì nam, nữ có đủ điều kiện kết hôn theo quy định của Luật này chung sống với nhau như vợ chồng mà không đăng k

In [7]:
len(df['train'])

31433

In [8]:
alpaca_dataset = []

for i in tqdm(range(len(df['train']))):
    element = df["train"][i]
    alpaca_dataset.append({
    "instruction": "Bạn là một chuyên gia về pháp luật Việt Nam với nhiều năm kinh nghiệm. Nhiệm vụ của bạn là cung cấp các câu trả lời chính xác, lập luận chặt chẽ, và xác thực. Hãy giúp mọi người giải đáp rõ ràng cho câu hỏi dưới đây:",
    "input": element["Question"],
    "output": element["Answer"],
    "system": "",
    "history": []
    })

100%|██████████| 31433/31433 [00:02<00:00, 11346.97it/s]


In [9]:
json.dump(alpaca_dataset, open('dataset/sample_train_alpaca_law_dataset.json', 'w'), ensure_ascii=False, indent=2)

In [12]:
alpaca_dataset = []

for i in tqdm(range(len(df['validation']))):
    element = df["validation"][i]
    alpaca_dataset.append({
    "instruction": "Bạn là một chuyên gia về pháp luật Việt Nam với nhiều năm kinh nghiệm. Nhiệm vụ của bạn là cung cấp các câu trả lời chính xác, lập luận chặt chẽ, và xác thực. Hãy giúp mọi người giải đáp rõ ràng cho câu hỏi dưới đây:",
    "input": element["Question"],
    "output": element["Answer"],
    "system": "",
    "history": []
    })

100%|██████████| 1746/1746 [00:00<00:00, 10129.73it/s]


In [13]:
json.dump(alpaca_dataset, open('dataset/sample_validation_alpaca_law_dataset.json', 'w'), ensure_ascii=False, indent=2)

In [14]:
alpaca_dataset = []

for i in tqdm(range(len(df['test']))):
    element = df["test"][i]
    alpaca_dataset.append({
    "instruction": "Bạn là một chuyên gia về pháp luật Việt Nam với nhiều năm kinh nghiệm. Nhiệm vụ của bạn là cung cấp các câu trả lời chính xác, lập luận chặt chẽ, và xác thực. Hãy giúp mọi người giải đáp rõ ràng cho câu hỏi dưới đây:",
    "input": element["Question"],
    "output": element["Answer"],
    "system": "",
    "history": []
    })

100%|██████████| 1747/1747 [00:00<00:00, 10417.97it/s]


In [15]:
json.dump(alpaca_dataset, open('dataset/sample_test_alpaca_law_dataset.json', 'w'), ensure_ascii=False, indent=2)

In [ ]:
!docker build -f ./docker/docker-cuda/Dockerfile \
    --build-arg INSTALL_BNB=false \
    --build-arg INSTALL_VLLM=false \
    --build-arg INSTALL_DEEPSPEED=true \
    --build-arg INSTALL_FLASHATTN=true \
    --build-arg PIP_INDEX=https://pypi.org/simple \
    -t llamafactory:latest .

In [ ]:
nvidia-docker run --name llamafactory-dai -it -v /data2/dai:/workspace/share --shm-size=64g -p 7860:7860 -p 8000:8000 llamafactory:latest

In [ ]:
nvidia-docker exec -it llamafactory-dai /bin/bash

In [ ]:
!torchrun src/train.py \
    --stage sft \
    --do_train \
    --use_fast_tokenizer \
    --flash_attn auto\
    --model_name_or_path Qwen2.5-7B-Instruct \
    --dataset train_alpaca_law \
    --template qwen \
    --finetuning_type lora \
    --lora_target q_proj,v_proj\
    --output_dir output \
    --overwrite_cache \
    --overwrite_output_dir \
    --warmup_steps 100 \
    --weight_decay 0.1 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 8 \
    --ddp_timeout 9000 \
    --learning_rate 5e-6 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --cutoff_len 2048 \
    --save_steps 1000 \
    --plot_loss \
    --num_train_epochs 1 \
    --save_total_limit 2 \
    --bf16 
    # \
    # --quantization_bit 8 \
    # --quantization_method bitsandbytes \
    
    # \
    # --resume_from_checkpoint output/checkpoint-24000


In [ ]:
!torchrun src/train.py \
    --stage sft \
    --do_train \
    --use_fast_tokenizer \
    --flash_attn auto\
    --model_name_or_path Qwen2.5-7B-Instruct \
    --dataset train_alpaca_law \
    --eval_dataset validation_alpaca_law \
    --template qwen \
    --finetuning_type lora \
    --lora_target q_proj,v_proj\
    --output_dir output \
    --overwrite_cache \
    --overwrite_output_dir \
    --warmup_steps 100 \
    --weight_decay 0.1 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --ddp_timeout 9000 \
    --learning_rate 5e-6 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --cutoff_len 4096 \
    --save_steps 1000 \
    --plot_loss \
    --num_train_epochs 3 \
    --bf16 \
    --quantization_bit 8 \
    --quantization_method bitsandbytes \
    --save_total_limit 1 \
    --resume_from_checkpoint output/checkpoint-24000


In [ ]:
CUDA_VISIBLE_DEVICES=0 llamafactory-cli train \
    --model_name_or_path Qwen2.5-7B-Instruct-fintune \
    --stage sft \
    --do_predict \
    --finetuning_type lora \
    --eval_dataset validation_alpaca_law \
    --template qwen \
    --cutoff_len 4096 \
    --max_samples 500000 \
    --overwrite_cache true \
    --preprocessing_num_workers 32 \
    --output_dir output/predict \
    --overwrite_output_dir true \
    --per_device_eval_batch_size 1 \
    --predict_with_generate true \
    --ddp_timeout 180000000 \
    --bf16 \
    --quantization_bit 8 \
    --quantization_method bitsandbytes 

In [ ]:
    # --adapter_name_or_path output/checkpoint-47148 \

In [ ]:
!CUDA_VISIBLE_DEVICES="0" llamafactory-cli export \
    --model_name_or_path Qwen2.5-7B-Instruct \
    --adapter_name_or_path output \
    --template qwen \
    --finetuning_type lora \
    --export_dir Qwen2.5-7B-Instruct-finetune-full \
    --export_size 4 \
    --export_legacy_format False

In [34]:
import glob
import json
import os
from urllib.parse import urlparse, unquote
from tqdm import tqdm

In [35]:
def get_filename_from_url(url):
    # Parse the URL to extract the path
    parsed_url = urlparse(url)
    # Decode non-ASCII characters
    path = unquote(parsed_url.path)
    # Extract the file name from the path
    filename = os.path.basename(path)
    output_name = '__SLASH__'.join([url.split('/')[-2], filename])
    return output_name

In [36]:
get_filename_from_url('https://congbobanan.toaan.gov.vn/5ta1506062t1cvn/21_2024_DS_GDT.pdf')

'5ta1506062t1cvn__SLASH__21_2024_DS_GDT.pdf'

In [37]:
files = glob.glob('dataset/vu_an/*/*')

In [38]:
len(files), files[:5]

(100000,
 ['dataset/vu_an/6/5ta1455906t1cvn__SLASH__sample_23.pdf',
  'dataset/vu_an/6/5ta1455908t1cvn__SLASH__sample_25.pdf',
  'dataset/vu_an/6/5ta1455909t1cvn__SLASH__Ban_an_ma_hoa.pdf',
  'dataset/vu_an/6/5ta1455912t1cvn__SLASH__ban_an_ma_hoa.pdf',
  'dataset/vu_an/6/5ta1455917t1cvn__SLASH__ban_an_ma_hoa.pdf'])

In [39]:
map_file_location = {file.split('/')[-1]: '/'.join(file.split('/')[-2:]) for file in files}
map_file_location['5ta1506062t1cvn__SLASH__21_2024_DS_GDT.pdf']

'0/5ta1506062t1cvn__SLASH__21_2024_DS_GDT.pdf'

In [40]:
'5ta1506062t1cvn__SLASH__21_2024_DS_GDT.pdf' in map_file_location

True

In [41]:
with open("/media/tavandai/DATA1/fpt_university/Graduation_Thesis/AIP491-G8-Graduation-Thesis/dataset/vu_an/metadata.txt") as f1:
    lines = [json.loads(line.strip()) for line in f1.readlines()]
lines[0]

{'title': 'Quyết định: số 21/2024/DS-GĐT ngày 25/04/2024 của Tòa án nhân dân tối cao (10.05.2024)',
 'url': 'https://congbobanan.toaan.gov.vn/2ta1506062t1cvn/chi-tiet-ban-an',
 'metadata': 'Tên quyết định: Tranh chấp quyền sử dụng đất, đòi lại tài sản, tranh chấp di sản thừa kế (15.07.2024)\nQuan hệ pháp luật:Tranh chấp đất đai\nCấp xét xử: Giám đốc thẩm\nLoại vụ/việc:Dân sự\nTòa án xét xử:Tòa án nhân dân tối cao\nÁp dụng án lệ: Không\nĐính chính: 0\nThông tin về vụ/việc: Tại đơn khởi kiện ngày 20/5/2014 của bà Lê Thu H và quá trình giải quyết vụ án, đại diện người kế thừa quyền, nghĩa vụ tố tụng của bà H là chị Trịnh Thị Mộng T trình bày: Phần đất có diện tích 1.390m2, tọa lạc tại ấp Vĩnh L, xã Vĩnh T, huyện Vĩnh T, thành phố Cần Thơ thuộc các thửa đất số 373 và 375, tờ bản đồ số 02, bà H được Ủy ban nhân dân huyện Thốt N (cũ) cấp Giấy chứng nhận quyền sử dụng ruộng đất ngày 25/4/1990 và cấp Giấy chứng nhận quyền sử dụng đất ngày 10/5/1996. Thửa đất số 374 của cụ Lê Thị N1 (bác ruột c

In [47]:
lines[100]

{'title': 'Quyết định: số 09/2023/DS-GĐT ngày 06/04/2023 của Tòa án nhân dân tối cao (21.04.2023)',
 'url': 'https://congbobanan.toaan.gov.vn/2ta1457007t1cvn/chi-tiet-ban-an',
 'metadata': 'Tên quyết định: Tranh chấp quyền sử dụng đất và yêu cầu hủy giấy chứng nhận quyền sử dụng đất (22.04.2024)\nQuan hệ pháp luật:Tranh chấp quyền sử dụng đất\nCấp xét xử: Giám đốc thẩm\nLoại vụ/việc:Dân sự\nTòa án xét xử:Tòa án nhân dân tối cao\nÁp dụng án lệ: Không\nĐính chính: 0\nThông tin về vụ/việc: Theo đơn khởi kiện đề ngày 11/4/2012 và các lời khai trong quá trình tố tụng, nguyên đơn là cụ Huỳnh Thị T (do ông Dương Hồng B là người đại diện theo ủy quyền, đồng thời là người có quyền lợi, nghĩa vụ liên quan) trình bày: Diện tích 1.838m2 đất đang tranh chấp (nằm trong diện tích khoảng 3.500m2 thuộc thửa cũ 252, 253, 200 tờ bản đồ số 08), nay thuộc thửa mới là 155a, 157a, 158, 159, 160 tờ bản đồ số 20 xã Phú H có nguồn gốc là của cố Huỳnh Thị B2 để lại cho cụ Huỳnh Thị T (mẹ của ông B). Do ông B sốn

In [42]:
for line in lines:
    if 'tối cao' in line['title']:
        pass
    elif 'cấp cao' in line['title']:
        pass
    elif 'tỉnh' in line['title']:
        pass
    elif 'TAND TP' in line['title']:
        pass
    elif 'TAND Q' in line['title']:
        pass
    elif 'Quân khu' in line['title']:
        pass
    elif 'Quân sự' in line['title'] or 'TAQS' in line['title']:
        pass
    elif "Thành phố " in line['title']:
        pass
    elif "TAND TX" in line['title']:
        pass
    elif "huyện" in line['title']:
        pass
    else:
        print(line)

In [61]:
metadata_dict = {}

for line in tqdm(lines):
    page_link = line['url']
    file_link = line['download_link']
    file_name = get_filename_from_url(file_link)
    if file_name in map_file_location:
        file_path = map_file_location[file_name]
    else:
        continue
    title = line['title']

    if 'tối cao' in title:
        cap_toa_an = 'tối cao'
    elif 'cấp cao' in title:
        cap_toa_an = 'cấp cao'
    elif 'tỉnh' in title:
        cap_toa_an = 'tỉnh'
    elif 'TAND TP' in title:
        cap_toa_an = 'thành phố'
    elif 'TAND Q' in title:
        cap_toa_an = 'quận'
    elif 'Quân khu' in title:
        cap_toa_an = 'quân khu'
    elif 'Quân sự' in title or 'TAQS' in title:
        cap_toa_an = 'quân sự'
    elif "Thành phố " in title:
        cap_toa_an = 'thành phố'
    elif "TAND TX" in title:
        cap_toa_an = 'thị xã'
    elif "huyện" in title:
        cap_toa_an = 'huyện'
    else:
        raise Exception

    metadata_dict[file_path] = {
        'page_link': page_link,
        'file_link': file_link,
        'file_name': file_name,
        'title': title,
        'cap_toa_an': cap_toa_an,
    }
    metadata = line['metadata'].split('\n')
    # print(metadata)
    for ele in metadata:
        # print(ele)
        sub_ele = ele.split(':')
        key = sub_ele[0].strip()
        value  = ':'.join(sub_ele[1:]).strip()
        metadata_dict[file_path][key] = value

100%|██████████| 102028/102028 [00:02<00:00, 41852.59it/s]


In [62]:
metadata_dict['0/5ta1506062t1cvn__SLASH__21_2024_DS_GDT.pdf']

{'page_link': 'https://congbobanan.toaan.gov.vn/2ta1506062t1cvn/chi-tiet-ban-an',
 'file_link': 'https://congbobanan.toaan.gov.vn/5ta1506062t1cvn/21_2024_DS_GDT.pdf',
 'file_name': '5ta1506062t1cvn__SLASH__21_2024_DS_GDT.pdf',
 'title': 'Quyết định: số 21/2024/DS-GĐT ngày 25/04/2024 của Tòa án nhân dân tối cao (10.05.2024)',
 'cap_toa_an': 'tối cao',
 'Tên quyết định': 'Tranh chấp quyền sử dụng đất, đòi lại tài sản, tranh chấp di sản thừa kế (15.07.2024)',
 'Quan hệ pháp luật': 'Tranh chấp đất đai',
 'Cấp xét xử': 'Giám đốc thẩm',
 'Loại vụ/việc': 'Dân sự',
 'Tòa án xét xử': 'Tòa án nhân dân tối cao',
 'Áp dụng án lệ': 'Không',
 'Đính chính': '0',
 'Thông tin về vụ/việc': 'Tại đơn khởi kiện ngày 20/5/2014 của bà Lê Thu H và quá trình giải quyết vụ án, đại diện người kế thừa quyền, nghĩa vụ tố tụng của bà H là chị Trịnh Thị Mộng T trình bày: Phần đất có diện tích 1.390m2, tọa lạc tại ấp Vĩnh L, xã Vĩnh T, huyện Vĩnh T, thành phố Cần Thơ thuộc các thửa đất số 373 và 375, tờ bản đồ số 02,

In [63]:
len(metadata_dict.keys())

99987

In [67]:
json.dump(metadata_dict, open('dataset/vu_an/metadata.json', 'w'), ensure_ascii=False, indent=2)

In [64]:
len(files)

100000

In [68]:
keys_a = list(metadata_dict.keys())
keys_b = list(map_file_location.values())

In [69]:
keys_a[0]

'0/5ta1506062t1cvn__SLASH__21_2024_DS_GDT.pdf'

In [70]:
keys_b[0]

'6/5ta1455906t1cvn__SLASH__sample_23.pdf'

In [72]:
missing = []
for ele in tqdm(keys_b):
    if ele not in keys_a:
        print(ele)
        missing.append(ele)

 12%|█▏        | 11764/100000 [00:44<05:32, 265.32it/s]

0/5ta1093892t1cvn__SLASH__QD_so_242021HCGDT_vu_Sinh_Hoa__UBND_thanh_pho_Vinh.pdf
0/5ta1093917t1cvn__SLASH__QD_so_352021HCGDT_vu_Nguyen_Thi_Kim.pdf


 13%|█▎        | 12805/100000 [00:47<02:12, 656.30it/s]

0/5ta1119000t1cvn__SLASH__QD_so_242021HCGDT_vu_Sinh_Hoa__UBND_thanh_pho_Vinh.pdf


 14%|█▍        | 14080/100000 [00:50<02:14, 639.90it/s]

...

 69%|██████▉   | 69351/100000 [03:38<01:28, 347.89it/s]

5/5ta1458323t1cvn__SLASH__58_2023_DSGDT.pdf
5/5ta1458324t1cvn__SLASH__59_2023_DSGDT.pdf


 70%|███████   | 70052/100000 [03:41<01:53, 265.02it/s]

7/Tran_Van_Manh.pdf
7/y_thng__h_cuong.pdf


100%|██████████| 100000/100000 [06:57<00:00, 239.33it/s]


In [73]:
missing_f = [ele for ele in missing if 'SLASH' in ele]

In [74]:
len(missing), len(missing_f)

(13, 11)

In [76]:
link = 'https://congbobanan.toaan.gov.vn/5ta1093892t1cvn/QD_so_242021HCGDT_vu_Sinh_Hoa__UBND_thanh_pho_Vinh.pdf'

In [77]:
for line in lines:
    if line['download_link'] == link:
        print(line)